Copyright (c) Microsoft Corporation. All rights reserved.

Licensed under the MIT License.

## Extractive Summarization on CNN/DM Dataset using Transformer Version of BertSum


### Summary

This notebook demonstrates how to fine tune Transformers for extractive text summarization. Utility functions and classes in the NLP Best Practices repo are used to facilitate data preprocessing, model training, model scoring, result postprocessing, and model evaluation.

BertSum refers to  [Fine-tune BERT for Extractive Summarization (https://arxiv.org/pdf/1903.10318.pdf) with [published example](https://github.com/nlpyang/BertSum/). And the Transformer version of Bertsum refers to our modification of BertSum and the source code can be accessed at (https://github.com/daden-ms/BertSum/). 

Extractive summarization are usually used in document summarization where each input document consists of mutiple sentences. The preprocessing of the input training data involves assigning label 0 or 1 to the document sentences based on the give summary. The summarization problem is also simplfied to classifying whether each document sentence should be included in the summary. 

The figure below illustrates how BERTSum can be fine tuned for extractive summarization task. Each sentence is inserted with [CLS] token at the beginning and  [SEP] at the end. Interval segment embedding and positional embedding are added upon the token embedding before input the BERT model. The [CLS] token representation is used as sentence embedding and only the [CLS] tokens are used as input for the summarization model. The summarization layer predicts whether the probability of each each sentence token should be included in the summary or not. Techniques like trigram blocking can be used to improve model accuarcy.   

<img src="https://nlpbp.blob.core.windows.net/images/BertSum.PNG">


### Before You Start

The running time shown in this notebook is on a Standard_NC24s_v3 Azure Deep Learning Virtual Machine with 4 NVIDIA Tesla V100 GPUs. 
> **Tip**: If you want to run through the notebook quickly, you can set the **`QUICK_RUN`** flag in the cell below to **`True`** to run the notebook on a small subset of the data and a smaller number of epochs. 

On a machine with 1 NVIDIA Tesla V100 GPUs, 16GB GPU memory configuration,
- for data preprocessing, it takes around 10 minutes the data preprocessing for quick run. Otherwise it takes ~2 hours to finish the data preprocessing. This time estimation assumes the chosen transformer model is "distilbert-base-uncased" and the sentence selection method is "greedy", which is the default. The preprocessing time can be significantly longer if the sentence selection method is "combination", which can achieve better model performance.

- for model fine tuning, it takes around 30 minutes for quick run. Otherwise, it takes around ~3 hours to finish. This estimation assume the chosen encoder method is "transformer". The model fine tuning time can be shorter if other encoder method is chosen, which may result in worse model performance. 


In [1]:
## Set QUICK_RUN = True to run the notebook on a small subset of data and a smaller number of epochs.
QUICK_RUN = True
## Set USE_PREPROCSSED_DATA = True to skip the data preprocessing
USE_PREPROCSSED_DATA = False

### Configuration

Before we start the notebook, we should set the environment variable to make sure you can access the GPUs on your machine

In [2]:
%load_ext autoreload

In [3]:
%autoreload 2

In [4]:
import os
import sys
from tempfile import TemporaryDirectory
import torch

nlp_path = os.path.abspath("../../")
if nlp_path not in sys.path:
    sys.path.insert(0, nlp_path)

from utils_nlp.common.pytorch_utils import get_device
from utils_nlp.dataset.cnndm import CNNDMBertSumProcessedData, CNNDMSummarization
from utils_nlp.eval.evaluate_summarization import get_rouge
from utils_nlp.models.transformers.extractive_summarization import (
    get_cycled_dataset,
    get_dataloader,
    get_sequential_dataloader,
    ExtractiveSummarizer,
    ExtSumProcessor,
)

[nltk_data] Downloading package punkt to /home/daden/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
I1213 19:06:40.821395 140049061730112 file_utils.py:39] PyTorch version 1.2.0 available.
I1213 19:06:40.854509 140049061730112 modeling_xlnet.py:194] Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex .


### Configuration: choose the transformer model to be used

In [5]:
# Transformer model being used
MODEL_NAME = "distilbert-base-uncased"

Also, we need to install the dependencies for pyrouge.

# dependencies for ROUGE-1.5.5.pl
!sudo apt-get update
!sudo apt-get install expat
!sudo apt-get install libexpat-dev -y

Run the following command in your terminal to install pre-requiste for using pyrouge.
1. sudo cpan install XML::Parser
1. sudo cpan install XML::Parser::PerlSAX
1. sudo cpan install XML::DOM

Download ROUGE-1.5.5 from https://github.com/andersjo/pyrouge/tree/master/tools/ROUGE-1.5.5.
Run the following command in your terminal.
* pyrouge_set_rouge_path $ABSOLUTE_DIRECTORY_TO_ROUGE-1.5.5.pl

### Data Preprossing

The dataset we used for this notebook is CNN/DM dataset which contains the documents and accompanying questions from the news articles of CNN and Daily mail. The highlights in each article are used as summary. The dataset consits of ~289K training examples, ~11K valiation and ~11K test dataset.  You can choose the [Option 1] below preprocess the data or [Option 2] to use the preprocessed version at [BERTSum published example](https://github.com/nlpyang/BertSum/). You don't need to manually download any of these two data sets as the code below will handle this part.  Since it takes up to 28 hours to preprocess the training data  to run on 10  Intel(R) Xeon(R) CPU E5-2690 v3 @ 2.60GHz, we suggest you continue with set as True first and experiment with data preprocessing  with QUICKRUN set as True.

##### Details of Data Preprocessing

The purpose of preprocessing is to process the input articles to the format that BertSum takes.  Functions defined specific in harvardnlp_cnndm_preprocess function are unique to CNN/DM dataset that's processed by harvardnlp. However, it provides a skeleton of how to preprocessing data into the format that BertSum takes. Assuming you have all articles and target summery each in a file, line-breaker seperated, the steps to preprocess the data are:
1. sentence tokenization
2. word tokenization
3. **label** the sentences in the article with 1 meaning the sentence is selected and 0 meaning the sentence is not selected. The options for the selection algorithms are "greedy" and "combination"
3. convert each example to  BertSum format
    - filter the sentences in the example based on the min_src_ntokens argument. If the lefted total sentence number is less than min_nsents, the example is discarded.
    - truncate the sentences in the example if the length is greater than max_src_ntokens
    - truncate the sentences in the example and the labels if the totle number of sentences is greater than max_nsents
    - [CLS] and [SEP] are inserted before and after each sentence
    - wordPiece tokenization
    - truncate the example to 512 tokens
    - convert the tokens into token indices corresponding to the BERT tokenizer's vocabulary.
    - segment ids are generated
    - [CLS] token positions are logged
    - [CLS] token labels are truncated if it's greater than 512, which is the maximum input length that can be taken by the BERT model.
    
    
Note that the original BERTSum paper use Stanford CoreNLP for data proprocessing, here we'll first how to use NLTK version, and then we also provide instruction of how to set up Stanford NLP and code examples of how to use Standford CoreNLP. 

##### [Option 1] Preprocess  data
The code in following cell will download the CNN/DM dataset listed at https://github.com/harvardnlp/sent-summary/.

In [6]:
# the data path used to save the downloaded data file
DATA_PATH = TemporaryDirectory().name
# The number of lines at the head of data file used for preprocessing. -1 means all the lines.
TOP_N = -1
if QUICK_RUN:
    TOP_N = 10000

In [7]:
train_dataset, test_dataset = CNNDMSummarization(top_n=TOP_N, local_cache_path=DATA_PATH)

100%|██████████| 489k/489k [00:08<00:00, 59.5kKB/s] 
I1213 19:06:49.528935 140049061730112 utils.py:173] Opening tar file /tmp/tmpuf604ztw/cnndm.tar.gz.


Preprocess the data and save the data to disk.

In [8]:
processor = ExtSumProcessor(model_name=MODEL_NAME)
ext_sum_train = processor.preprocess(train_dataset, train_dataset.get_target(), oracle_mode="greedy")
ext_sum_test = processor.preprocess(test_dataset, test_dataset.get_target(),oracle_mode="greedy")

I1213 19:07:00.743392 140049061730112 tokenization_utils.py:374] loading file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-vocab.txt from cache at ./26bc1ad6c0ac742e9b52263248f6d0f00068293b33709fae12320c0e35ccfbbb.542ce4285a40d23a559526243235df47c5f75c197f04f37d1a0c124c32c9a084


{'max_nsents': 200, 'max_src_ntokens': 2000, 'min_nsents': 3, 'min_src_ntokens': 5, 'use_interval': True}


In [9]:
save_path = os.path.join(DATA_PATH, "processed")
train_files = CNNDMBertSumProcessedData.save_data(
    ext_sum_train, is_test=False, save_path=save_path, chunk_size=2000
)
test_files = CNNDMBertSumProcessedData.save_data(
    ext_sum_test, is_test=True, save_path=save_path, chunk_size=2000
)

In [10]:
train_files

['/tmp/tmpuf604ztw/processed/0_train',
 '/tmp/tmpuf604ztw/processed/1_train',
 '/tmp/tmpuf604ztw/processed/2_train',
 '/tmp/tmpuf604ztw/processed/3_train',
 '/tmp/tmpuf604ztw/processed/4_train']

In [11]:
test_files

['/tmp/tmpuf604ztw/processed/0_test',
 '/tmp/tmpuf604ztw/processed/1_test',
 '/tmp/tmpuf604ztw/processed/2_test',
 '/tmp/tmpuf604ztw/processed/3_test',
 '/tmp/tmpuf604ztw/processed/4_test']

In [12]:
train_dataset_generator, test_dataset_generator = CNNDMBertSumProcessedData().splits(root=save_path)

#### Inspect Data

In [13]:
import torch
bert_format_data = torch.load(train_files[0])
print(len(bert_format_data))
bert_format_data[0].keys()

2000


dict_keys(['src', 'labels', 'segs', 'clss', 'src_txt', 'tgt_txt'])

In [14]:
bert_format_data[0]['labels']

[0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

##### [Option 2] Reuse Preprocessed  data from [BERTSUM Repo](https://github.com/nlpyang/BertSum)

In [15]:
# the data path used to downloaded the preprocessed data from BERTSUM Repo.
# if you have downloaded the dataset, change the code to use that path where the dataset is.
PROCESSED_DATA_PATH = TemporaryDirectory().name
#data_path = "./temp_data5/"
#PROCESSED_DATA_PATH = data_path

In [16]:
if USE_PREPROCSSED_DATA:
    CNNDMBertSumProcessedData.download(local_path=PROCESSED_DATA_PATH)
    train_dataset_generator, test_dataset_generator = CNNDMBertSumProcessedData().splits(root=PROCESSED_DATA_PATH)
    

### Model training
To start model training, we need to create a instance of ExtractiveSummarizer.
#### Choose the transformer model.
Currently ExtractiveSummarizer support two models:
- distilbert-base-uncase, 
- bert-base-uncase

Potentionally, roberta-based model and xlnet can be supported but needs to be tested.
#### Choose the encoder algorithm.
There are four options:
- baseline: it used a smaller transformer model to replace the bert model and with transformer summarization layer
- classifier: it uses pretrained BERT and fine-tune BERT with **simple logistic classification** summarization layer
- transformer: it uses pretrained BERT and fine-tune BERT with **transformer** summarization layer
- RNN: it uses pretrained BERT and fine-tune BERT with **LSTM** summarization layer

In [17]:
# notebook parameters
# the cache data path during find tuning
CACHE_DIR = TemporaryDirectory().name

# batch size, unit is the number of tokens
BATCH_SIZE = 3000

# GPU used for training
NUM_GPUS = 1

# Encoder name. Options are: 1. baseline, classifier, transformer, rnn.
ENCODER = "transformer"

# Learning rate
LEARNING_RATE=2e-3

# How often the statistics reports show up in training, unit is step.
REPORT_EVERY=100
    
if QUICK_RUN:
    # total number of steps for training
    MAX_STEPS=1e4
    # number of steps for warm up
    WARMUP_STEPS=5e3
    
else:
    MAX_STEPS=5e4
    WARMUP_STEPS=5e3
 

In [18]:
summarizer = ExtractiveSummarizer(MODEL_NAME, ENCODER, CACHE_DIR)

I1213 19:14:40.413720 140049061730112 file_utils.py:296] https://s3.amazonaws.com/models.huggingface.co/bert/distilbert-base-uncased-config.json not found in cache or force_download set to True, downloading to /tmp/tmpvow1cz9j
100%|██████████| 492/492 [00:00<00:00, 544628.55B/s]
I1213 19:14:40.574677 140049061730112 file_utils.py:309] copying /tmp/tmpvow1cz9j to cache at /tmp/tmp1x0h8exo/a41e817d5c0743e29e86ff85edc8c257e61bc8d88e4271bb1b243b6e7614c633.1ccd1a11c9ff276830e114ea477ea2407100f4a3be7bdc45d37be9e37fa71c7e
I1213 19:14:40.575870 140049061730112 file_utils.py:313] creating metadata file for /tmp/tmp1x0h8exo/a41e817d5c0743e29e86ff85edc8c257e61bc8d88e4271bb1b243b6e7614c633.1ccd1a11c9ff276830e114ea477ea2407100f4a3be7bdc45d37be9e37fa71c7e
I1213 19:14:40.577564 140049061730112 file_utils.py:322] removing temp file /tmp/tmpvow1cz9j
I1213 19:14:40.578326 140049061730112 configuration_utils.py:151] loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/distilbert

In [19]:
# batch_size is the number of tokens in a batch
train_dataloader = get_dataloader(get_cycled_dataset(train_dataset_generator), is_labeled=True, batch_size=3000)

In [20]:
summarizer.fit(
            train_dataloader,
            num_gpus=1,
            gradient_accumulation_steps=2,
            max_steps=MAX_STEPS,
            lr=LEARNING_RATE,
            warmup_steps=WARMUP_STEPS,
            verbose=True,
            report_every=REPORT_EVERY,
            clip_grad_norm=False,
        )

loss: 34.033510, time: 20.922801, number of examples in current step: 5, step 100 out of total 10000
loss: 32.287387, time: 21.290814, number of examples in current step: 5, step 200 out of total 10000
loss: 31.838500, time: 21.051537, number of examples in current step: 5, step 300 out of total 10000
loss: 31.183838, time: 21.305765, number of examples in current step: 5, step 400 out of total 10000
loss: 30.655987, time: 20.895105, number of examples in current step: 6, step 500 out of total 10000
loss: 30.956138, time: 21.488802, number of examples in current step: 8, step 600 out of total 10000
loss: 30.789554, time: 20.859549, number of examples in current step: 5, step 700 out of total 10000
loss: 30.939141, time: 21.222007, number of examples in current step: 5, step 800 out of total 10000
loss: 30.246895, time: 20.882249, number of examples in current step: 5, step 900 out of total 10000
loss: 30.127477, time: 21.238788, number of examples in current step: 5, step 1000 out of t

loss: 7.357592, time: 21.266071, number of examples in current step: 5, step 8200 out of total 10000
loss: 6.336736, time: 20.806072, number of examples in current step: 5, step 8300 out of total 10000
loss: 5.877894, time: 21.371235, number of examples in current step: 5, step 8400 out of total 10000
loss: 5.964879, time: 20.897553, number of examples in current step: 5, step 8500 out of total 10000
loss: 5.535290, time: 21.059212, number of examples in current step: 5, step 8600 out of total 10000
loss: 6.686879, time: 20.794497, number of examples in current step: 5, step 8700 out of total 10000
loss: 5.563559, time: 21.314857, number of examples in current step: 5, step 8800 out of total 10000
loss: 4.289995, time: 20.640453, number of examples in current step: 5, step 8900 out of total 10000
loss: 4.875267, time: 21.064786, number of examples in current step: 2, step 9000 out of total 10000
loss: 5.190399, time: 20.760948, number of examples in current step: 5, step 9100 out of to

In [21]:
summarizer.save_model("extsum_modelname_{0}_quickrun_{1}.pt".format(MODEL_NAME, QUICK_RUN))

I1213 19:49:55.251679 140049061730112 extractive_summarization.py:432] Saving model checkpoint to /tmp/tmp1x0h8exo/fine_tuned/extsum_modelname_distilbert-base-uncased_quickrun_True.pt


In [22]:
# for loading a previous saved model
#import torch
#summarizer.model = torch.load("cnndm_transformersum_distilbert-base-uncased_bertsum_processed_data.pt")

### Model Evaluation

[ROUGE](https://en.wikipedia.org/wiki/ROUGE_(metric)), or Recall-Oriented Understudy for Gisting Evaluation has been commonly used for evaluation text summarization.

In [23]:
test_dataset=[]
for i in test_dataset_generator():
    test_dataset.extend(i)
target = [test_dataset[i]['tgt_txt'] for i in range(len(test_dataset))]

In [24]:
prediction = summarizer.predict(get_sequential_dataloader(test_dataset))
#prediction = summarizer.predict(get_dataloader(test_dataset_generator()))

In [25]:
rouge_transformer = get_rouge(prediction, target, "./results/")

9999
9999


2019-12-13 19:51:06,253 [MainThread  ] [INFO ]  Writing summaries.
I1213 19:51:06.253292 140049061730112 pyrouge.py:525] Writing summaries.
2019-12-13 19:51:06,264 [MainThread  ] [INFO ]  Processing summaries. Saving system files to ./results/tmpe55w9_ms/system and model files to ./results/tmpe55w9_ms/model.
I1213 19:51:06.264005 140049061730112 pyrouge.py:518] Processing summaries. Saving system files to ./results/tmpe55w9_ms/system and model files to ./results/tmpe55w9_ms/model.
2019-12-13 19:51:06,264 [MainThread  ] [INFO ]  Processing files in ./results/rouge-tmp-2019-12-13-19-51-05/candidate/.
I1213 19:51:06.264976 140049061730112 pyrouge.py:43] Processing files in ./results/rouge-tmp-2019-12-13-19-51-05/candidate/.
2019-12-13 19:51:07,212 [MainThread  ] [INFO ]  Saved processed files to ./results/tmpe55w9_ms/system.
I1213 19:51:07.212478 140049061730112 pyrouge.py:53] Saved processed files to ./results/tmpe55w9_ms/system.
2019-12-13 19:51:07,214 [MainThread  ] [INFO ]  Processing

---------------------------------------------
1 ROUGE-1 Average_R: 0.48184 (95%-conf.int. 0.47899 - 0.48488)
1 ROUGE-1 Average_P: 0.34422 (95%-conf.int. 0.34185 - 0.34659)
1 ROUGE-1 Average_F: 0.38649 (95%-conf.int. 0.38434 - 0.38879)
---------------------------------------------
1 ROUGE-2 Average_R: 0.20180 (95%-conf.int. 0.19931 - 0.20468)
1 ROUGE-2 Average_P: 0.14448 (95%-conf.int. 0.14249 - 0.14667)
1 ROUGE-2 Average_F: 0.16185 (95%-conf.int. 0.15983 - 0.16404)
---------------------------------------------
1 ROUGE-L Average_R: 0.43652 (95%-conf.int. 0.43387 - 0.43948)
1 ROUGE-L Average_P: 0.31246 (95%-conf.int. 0.31003 - 0.31479)
1 ROUGE-L Average_F: 0.35053 (95%-conf.int. 0.34841 - 0.35273)



In [26]:
test_dataset[0]['tgt_txt']

'marseille prosecutor says `` so far no videos were used in the crash investigation `` despite media reports .<q>journalists at bild and paris match are `` very confident `` the video clip is real , an editor says .<q>andreas lubitz had informed his lufthansa training school of an episode of severe depression , airline says .<q>'

In [27]:
prediction[0]

'marseille , france ( cnn ) the french prosecutor leading an investigation into the crash of germanwings flight 9525 insisted wednesday that he was not aware of any video footage from on board the plane .<q>marseille prosecutor brice robin told cnn that `` so far no videos were used in the crash investigation . ``<q>paris match and bild reported that the video was recovered from a phone at the wreckage site .'

In [28]:
test_dataset[0]['src_txt']

['marseille , france ( cnn ) the french prosecutor leading an investigation into the crash of germanwings flight 9525 insisted wednesday that he was not aware of any video footage from on board the plane .',
 'marseille prosecutor brice robin told cnn that `` so far no videos were used in the crash investigation . ``',
 'he added , `` a person who has such a video needs to immediately give it to the investigators . ``',
 "robin 's comments follow claims by two magazines , german daily bild and french paris match , of a cell phone video showing the harrowing final seconds from on board germanwings flight 9525 as it crashed into the french alps .",
 'all 150 on board were killed .',
 'paris match and bild reported that the video was recovered from a phone at the wreckage site .',
 'the two publications described the supposed video , but did not post it on their websites .',
 'the publications said that they watched the video , which was found by a source close to the investigation . ``',